In [1]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Cover Letter.txt to Cover Letter.txt


In [3]:
text = open('Cover Letter.txt', encoding='UTF-8').read().lower()
print('Corpus Length:',len(text))

Corpus Length: 264


In [5]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [6]:
unique_words = np.unique(words)
unique_words_index = dict((c,i)for i, c in enumerate(unique_words))

In [7]:
WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
  prev_words.append(words[i:i + WORD_LENGTH])
  next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['honestly', 'i', 'possess', 'all', 'the']
skills


In [10]:
X = np.zeros((len(prev_words),WORD_LENGTH, len(unique_words)), dtype = bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
  for j,each_word in enumerate(each_words):
      X[i,j, unique_words_index[each_word]] = 1
  Y[i, unique_words_index[next_words[i]]] = 1

In [11]:
print(X[0][0])

[False False False False False False False False False False False False
 False  True False False False False False False False False False False
 False False False False False False False False False False False False
 False False False]


In [12]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH,len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

In [13]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history

Epoch 1/2


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


1/1 [==============================] - 3s 3s/step - loss: 3.6645 - accuracy: 0.0244 - val_loss: 3.6818 - val_accuracy: 0.0000e+00
Epoch 2/2
1/1 [==============================] - 0s 37ms/step - loss: 3.3747 - accuracy: 0.5366 - val_loss: 3.8707 - val_accuracy: 0.0000e+00


In [14]:
model.save('keras_next_word_model.h5')
pickle.dump(history, open("history.p","wb"))
model = load_model('keras_next_word_model.h5')
history = pickle.load(open("history.p","rb"))

In [20]:
def prepare_input(text):
  x=np.zeros((1,WORD_LENGTH,len(unique_words)))
  for t, word in enumerate(text.split()):
    print(word)
    x[0, t, unique_words_index[word]] = 1
  return x
prepare_input("I possess all the skills".lower())

i
possess
all
the
skills


array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0.]]])

In [22]:
def sample(preds, top_n=3):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds)
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)

  return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [23]:
def predict_completion(text, n=3):
  if text =="":
    return("0")
  x = prepare_input(text)
  preds = model.predict(x, verbose=0)[0]
  next_indices = sample(preds, n)
  return[unique_words[idx] for idx in next_indices]

In [24]:
m = "I'm pretty confident that I am the best candidate for this job role"
print("correct sentences: ",m)
seq = " ".join(tokenizer.tokenize(m.lower())[0:5])
print("Sequence: ",seq)
print("Next possible word: ", predict_completion(seq, 5))

correct sentences:  I'm pretty confident that I am the best candidate for this job role
Sequence:  i m pretty confident that
i
m
pretty
confident
that
Next possible word:  ['i', 'the', 'best', 'that', 'am']
